## Method 2: Tucker3 decomposition:
We can naturally treat panel data as a three-dimensional tensor for processing. The essence of Tucker decomposition of a tensor is to decompose the original tensor into a core tensor and corresponding factor matrices for different dimensions. Each factor matrix can be considered as a linear transformation operation corresponding to different dimensions. The core tensor is akin to a compressed form of the original tensor, that is, the result of dimensionality reduction on the original tensor. Therefore, Tucker decomposition can be regarded as a higher-order form of PCA dimensionality reduction.

### trying tucker3 decomposition using R

In [4]:
library(readxl)
library(imputeTS)
library(dplyr)
library(tidyr)
library(naniar)

data<-read_excel('Treatment-SolidWaste.xlsx')

Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




* Group the dataset by region and interpolate within each region by exponential weighting moving average.

In [5]:
df1<-data %>%
    group_by(Region) %>%
        mutate(across(c(Output,Utilization,Storage,Disposition),~na_ma(.x, k = 2, weighting = "exponential", maxgap = Inf)))%>%
            ungroup()

# check data from 2007-2010, and 2018-2019
selected_years <- c(2007, 2008, 2009, 2010, 2018, 2019)
subset_data <- df1[df1$Year %in% selected_years, ]
head(subset_data)

Region,Year,Output,Utilization,Storage,Disposition
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Beijing,2019,455.180,262.0813,1.715,193.3312
Tianjin,2019,1811.885,1764.3150,3.560,46.0650
Hebei,2019,36108.645,19618.3488,8124.678,8632.8487
Shanxi,2019,43905.298,17414.9788,6159.765,20687.7288
Neimenggu,2019,37076.500,12222.7650,11188.667,14790.0150
Liaoning,2019,24915.460,11746.4163,6337.318,7447.6662


* Turn the dataset into a tensor

In [6]:
unique_regions <- unique(df1$Region)
unique_years <- unique(df1$Year)
unique_indicators <- colnames(df1)[3:ncol(df1)]  

n <- length(unique_regions)
m <- length(unique_indicators)
r <- length(unique_years)

# create a new empty tensor
tensor <- array(NA, dim = c(n, m, r))

# fill the empty tensor with data
for (i in 1:n) {
  for (j in 1:m) {
    for (k in 1:r) {
      # select data according to 'Region', 'Year' and other indicators
      value <- as.numeric(df1[df1$Region == unique_regions[i] & df1$Year == unique_years[k], unique_indicators[j]])
      if (length(value) == 1) {
        tensor[i, j, k] <- value
      }
    }
  }
}

dims <- dim(tensor)
print(dims)

# define a function in order to select data by [Region, Year,indicator]
select_data <- function(region, year, indicator) {
  i <- which(unique_regions == region)
  j <- which(unique_indicators == indicator)
  k <- which(unique_years == year)
  return(tensor[i, j, k])
}

# an example
result <- select_data("Beijing", "2007", "Output")
print(result)

[1] 31  4 19
[1] 1316.667


## Step 1: perform dual-centralized preprocessing on $X(n\times m\times r)$(ij,jt centering), and then perform one-way standardization on the variables to generate a new data $X_{new}(n\times m\times r)$.

In [7]:
# 1.calculate the means of ij slice and jk slice.
ij_mean <- apply(tensor, c(1,2), mean, na.rm = TRUE) # i by j matrix of means across k
jk_mean <- apply(tensor, c(2,3), mean, na.rm = TRUE) # j by k matrix of means across i

# 2.let each data substract the means of the two slices. 
for (i in 1:n) {
  for (j in 1:m) {
    for (k in 1:r) {
      tensor[i,j,k] <- tensor[i,j,k] - ij_mean[i,j] - jk_mean[j,k]
    }
  }
}

# 3.calculate the mean and standard deviation of ik slice
ik_mean <- apply(tensor, c(1,3), mean, na.rm = TRUE) # i by k matrix of means across j
ik_sd <- apply(tensor, c(1,3), sd, na.rm = TRUE)    # i by k matrix of standard deviations across j

# 4.standardize each data in the direction of the variable
for (i in 1:n) {
  for (j in 1:m) {
    for (k in 1:r) {
      tensor[i,j,k] <- (tensor[i,j,k] - ik_mean[i,k])/ik_sd[i,k] 
    }
  }
}

dims <-dim(tensor)
print(dims)

# for instance
result_standard <- select_data("Beijing", "2007", "Output")
print(result_standard)
print(tensor[])

[1] 31  4 19
[1] -1.320413
, , 1

            [,1]        [,2]        [,3]       [,4]
 [1,] -1.3396661 -0.18946814  0.75293395  0.7762003
 [2,] -1.3705199 -0.11684487  0.71289520  0.7744695
 [3,] -0.5256922  0.48447059  1.13597763 -1.0947561
 [4,]  0.6268851 -1.03764813 -0.64489440  1.0556575
 [5,]  0.8394386 -1.41267947  0.02645029  0.5467906
 [6,] -1.4741242  0.58603460  0.23289373  0.6551959
 [7,] -1.3144406 -0.23957681  0.71632943  0.8376879
 [8,] -1.1826673 -0.44866351  1.00191674  0.6294140
 [9,] -1.3518005 -0.16072706  0.71797204  0.7945555
[10,] -1.4710525  0.25019747  0.49176390  0.7290912
[11,] -1.4279597  0.04468924  0.65835348  0.7249170
[12,] -1.4889003  0.66602343  0.38879594  0.4340810
[13,] -1.4236565  0.03126978  0.73396944  0.6584173
[14,] -1.3361082 -0.12767031  0.98407110  0.4797074
[15,] -1.3097904 -0.15149309  1.04194873  0.4193348
[16,] -1.4368756  0.36894593  0.87236596  0.1955637
[17,] -1.3423622 -0.18358439  0.77017823  0.7557684
[18,] -1.3739534 -0.10898712  

* transform the data into the form that python recognizes, just in case the R package doesn't work, for the R package rTensor is recently released. 

In [10]:
print_tensor <- function(tensor) {
  str <- ""
  for (k in 1:dim(tensor)[3]) {
    str <- paste0(str, "[")
    for (i in 1:dim(tensor)[1]) {
      str <- paste0(str, "[")
      for (j in 1:dim(tensor)[2]) {
        str <- paste0(str, tensor[i, j, k])
        if (j < dim(tensor)[2]) {
          str <- paste0(str, ", ")
        }
      }
      str <- paste0(str, "]")
      if (i < dim(tensor)[1]) {
        str <- paste0(str, ", ")
      }
    }
    str <- paste0(str, "]")
    if (k < dim(tensor)[3]) {
      str <- paste0(str, ", ")
    }
  }
  cat(str)
}

In [11]:
print_tensor(tensor)

[[-1.31137701395777, -0.2482367740501, 0.744655454486734, 0.814958333521139], [-1.3604951671379, -0.137799282543023, 0.68741745506648, 0.810876994614447], [-1.17198979906625, -0.0745247362338451, 1.27286743198583, -0.02635289668573], [-0.822834478252547, -0.801520202258153, 0.387180444547627, 1.23717423596307], [-0.742450945150766, -0.975988779358858, 0.759985409507834, 0.95845431500179], [-1.4850912226971, 0.400575074049512, 0.390673061905715, 0.693843086741869], [-1.27649790667161, -0.306165424384071, 0.680868510040132, 0.901794821015545], [-1.11604811779207, -0.544544673155281, 1.0291977391567, 0.631395051790648], [-1.32948894040823, -0.206129314374291, 0.69395741489547, 0.841660839887052], [-1.45606246259738, 0.180024438880631, 0.508912878318726, 0.767125145398018], [-1.42714449579832, 0.0457832127704768, 0.631170186899548, 0.750191096128293], [-1.49513299342415, 0.386317307190944, 0.537085067345076, 0.571730618888132], [-1.42136724273152, 0.0223412880479635, 0.716530867913839, 0.6

In [8]:
# mean() function doesn't work(seemingly) after adjusting its format in the next cell, so we calculate it here. 
mean_tensor <- mean(tensor)
mean_tensor

[1] -1.215954e-18

## Step 2:
### Conduct Tucker3 analysis with $X_{new}(n\times m \times r)$ .The component load constraint of the three modes is orthogonal, and the component decomposition dimensions P,Q,S of the model are calculated according to the $\overline{R}^2$ under the premise of ensuring the interpretability of the model

Tucker3 model:
$$
x_{ijt} = \sum_{p=1}^{P} \sum_{q=1}^{Q} \sum_{s=1}^{S} b_{jq}c_{ts}g_{pqs} + e_{ijt}                
$$
and in a matrix format:
$$
X_{a} = AG_{a}(C'\otimes B') + E_{a} 
$$

In [9]:
library(rTensor)
tensor <-as.tensor(tensor)
best_fit <- NULL
best_percent <- -Inf

for(p in 1:n) {
  for(q in 1:m) {
    for(s in 1:r) {
      result <- tucker(tensor, ranks = c(p, q, s), max_iter = 25, tol = 1e-05) 
      resid_2 <- (result$fnorm_resid)^2
      tensor_data <- tensor@data  
      total_var <- sum((tensor_data - mean_tensor)^2)
      R_2 <- 1 - resid_2 / total_var
      N <- prod(dim(tensor_data))
      k <- sum(sapply(result$U, nrow)) + length(result$Z) - 1
      R_2_adj <- 1 - ((1 - R_2) * (N - 1) / (N - k - 1))
      
      if (is.numeric(R_2_adj) && length(R_2_adj) == 1 && !is.na(R_2_adj)) {
        if (R_2_adj > best_percent) {
          best_percent <- R_2_adj
          best_fit <- result
        }
      }
    }
  }
}
best_percent
best_fit

  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |=====================================

[1] 1

$Z
Numeric Tensor of 3 Modes
Modes:  31 3 19 
Data: 
, , 1

            [,1]       [,2]       [,3]
[1,] 39.74249010  0.3621438 -0.4317076
[2,] -0.02103769 -0.3841103  0.1076564
[3,] -0.11109628  0.3540906 -0.1856649
[4,] -0.10764379 -0.2043494 -0.1030666
[5,]  0.01504067 -0.4720666  0.3102244
[6,]  0.02637305  0.4957841 -0.5655761

, , 2

            [,1]       [,2]       [,3]
[1,] -0.02143854 -0.3789063  0.5618420
[2,]  2.88601158 -4.2684651  4.9268753
[3,]  0.47673375 -3.7099116 -1.3483234
[4,]  0.53741755  1.6548698  2.9778822
[5,] -0.52962334 -1.2183003 -1.1707670
[6,]  1.26456923  0.9953209 -0.4809639

, , 3

            [,1]       [,2]        [,3]
[1,]  0.04833258 -0.1930716 -0.66695684
[2,] -0.12834193  1.4999492  1.99852345
[3,] -1.66962561 -2.1185056 -1.18302248
[4,] -1.55628334  0.3876316 -2.04841951
[5,] -0.56983080  3.3880914  0.04081858
[6,] -0.68746286  0.7372266  1.37000154

, , 4

            [,1]       [,2]        [,3]
[1,]  0.02481175 -0.7333355 -0.06430941
[2,] -0.77

It is clear that the Tucker3 decomposition by R package does not oresent the effect we pursue, because the core tensor's dimensions [31,3,19] only have slight differences with the tensor's [31,4,19].Since the R package is quite new, I'll turn to relevant tools in python for help.